In [3]:
from collections import defaultdict
import heapq
import numpy as np

In [4]:
SMALL_MATRIX = 'small_matrix.txt'
LARGE_MATRIX = 'large_matrix.txt'
UP, DOWN, LEFT, RIGHT = (-1,0), (1,0), (0,-1), (0,1)

In [5]:
def read_file(filename):
    matrix = []
    with open(filename) as file:
        for line in file:
            matrix.append([int(x) for x in line.split(',')])
    return np.asarray(matrix)

# Solution for 081-082-083 with Dijkstra

In [6]:
def get_neighbours(i, j, directions, matrix):
    neighbours = []
    for di, dj in directions:
        try:
            neighbours.append((matrix[i+di][j+dj], (i+di, j+dj)))
        except IndexError:
            continue
    return neighbours

In [7]:
# Matrix implementation of dijkstra
def dijkstra(matrix, source, target, directions):
    # Create data structures needed
    queue = [(matrix[source],source,())] # nodes need to process
    seen = set() # set (O(1) lookup) of seen nodes
    mins = {source: 0} # dict with min path dist from source
    
    if not isinstance(target, set):
        target = {target}
    
    while queue:
        # Pop node in queue with smallest cost
        cost, v1, path = heapq.heappop(queue) # num, (i, j), pathlist
        
        # Process it if not seen yet
        if v1 not in seen:
            seen.add(v1) # add as seen
            path = (v1, *path) # and update path
            
            # Finished if v1 is target
            if v1 in target: 
                return (cost, path)

            # If not, we need to process v1's neighbours
            for c, v2 in get_neighbours(*v1, directions, matrix):
                
                # Only process if not seen before
                if v2 not in seen:
                    
                    prev = mins.get(v2, None)
                    nextt = cost + c
                    
                    if prev is None or nextt < prev:
                        mins[v2] = nextt
                        heapq.heappush(queue, (nextt, v2, path))

    return (-1, ())

In [8]:
def solve_p81():
    matrix = read_file(LARGE_MATRIX)
    l = len(matrix) - 1
    return dijkstra(matrix, (0,0), (l,l), (RIGHT, DOWN))

print(solve_p81)
%timeit solve_p81()

42 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
def solve_p82():
    matrix = read_file(LARGE_MATRIX)
    l = len(matrix) - 1
    targets = {(idx,l) for idx in range(len(matrix))}
    return min([dijkstra(matrix, (idx,0), targets, (RIGHT, UP, DOWN)) for idx in range(len(matrix))])
    
print(solve_p82()[0])
%timeit solve_p82()

260324
3.83 s ± 51.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
def solve_p83():
    matrix = read_file(LARGE_MATRIX)
    l = len(matrix) - 1
    return dijkstra(matrix, (0,0), (l,l), (RIGHT, LEFT, UP, DOWN))

print(solve_p83()[0])
%timeit solve_p83()

425185
246 ms ± 8.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Solution for p081, calculating min path through matrix

In [13]:
def find_min_path(matrix):
    cost_matrix = np.zeros(shape=(len(matrix), len(matrix)))
    cost_matrix[0][0] = matrix[0][0]
    
    # Start with finding cost of first row and first column
    for i in range(1,len(matrix)):
        cost_matrix[0][i] = cost_matrix[0][i-1] + matrix[0][i]
        cost_matrix[i][0] = cost_matrix[i-1][0] + matrix[i][0]
    
    # Then loop rest of the two matrixes...
    for i in range(1, len(matrix)):
        for j in range(1, len(matrix)):
            cost_matrix[i][j] = min(cost_matrix[i-1][j], cost_matrix[i][j-1]) + matrix[i][j]
            
    return cost_matrix

def solve():
    matrix = read_file(LARGE_MATRIX)
    return int(find_min_path(matrix)[-1][-1])

%timeit solve()

14.8 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Solution for p082, using edges and graph in dictionary instead of using matrix + get_neigbours()

In [20]:
def create_edges(matrix):
    num_to_idx = {}
    idx_to_num = {}
    edges = []
    
    # Create a mapping from unique number to idx, and vice versa
    counter = 1
    for idx, x in enumerate(matrix):
        for idy, y in enumerate(x):
            num_to_idx[counter] = (idx, idy)
            idx_to_num[(idx,idy)] = counter
            counter += 1
    
    # Create all edges pointing to the right
    for i in range(len(matrix)):
        for j in range(len(matrix)-1):
            f = idx_to_num[i,j]
            t = idx_to_num[i,j+1]
            c = matrix[i,j+1]
            edges.append((f,t,c))
    
    # Create all edges pointing down
    for i in range(len(matrix) - 1):
        for j in range(1, len(matrix)-1):
            f = idx_to_num[i,j]
            t = idx_to_num[i+1,j]
            c = matrix[i+1,j]
            edges.append((f,t,c))
            
    # Create all edges pointing up
    for i in range(1, len(matrix)):
        for j in range(1, len(matrix)-1):
            f = idx_to_num[i,j]
            t = idx_to_num[i-1,j]
            c = matrix[i-1,j]
            edges.append((f,t,c))
            
    # And lastly create two extra edges for start and end:
    for i in range(len(matrix)):
        edges.append((0, idx_to_num[i, 0], matrix[i][0]))
        edges.append((idx_to_num[i,len(matrix)-1], -1, 0))
        
    return edges

In [35]:
# Graph implementation of dijkstra
def dijkstra_graph(edges, source, target):
    g = defaultdict(list)
    for l,r,c in edges:
        g[l].append((c,r))

    # Create data structures needed
    queue = [(0,source,())] # nodes need to process
    seen = set() # set (O(1) lookup) of seen nodes
    mins = {source: 0} # dict with min path dist from source
    
    while queue:
        # Pop node in queue with smallest cost
        cost, v1, path = heapq.heappop(queue) # num, (i, j), pathlist
        
        # Process it if not seen yet
        if v1 not in seen:
            seen.add(v1) # add as seen
            path = (v1, *path) # and update path
            
            # Finished if v1 is target
            if v1 == target: 
                return (cost, path)

            # If not, we need to process v1's neighbours
            for c, v2 in g.get(v1,None):
                
                # Only process if not seen before
                if v2 not in seen:
                    prev = mins.get(v2, None)
                    nextt = cost + c
                    
                    if prev is None or nextt < prev:
                        mins[v2] = nextt
                        heapq.heappush(queue, (nextt, v2, path))

    return (-1, ())

def solve_p82_graph():
    return dijkstra_graph(create_edges(read_file(LARGE_MATRIX)), 0, -1)[0]
    

In [36]:
print(solve_p82_graph())
%timeit solve_p82_graph()

260324
39.2 ms ± 859 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
